In [1]:
import torch   #查看torch版本
print(torch.__version__)  #注意是双下划线

import torchvision  #查看torchvision版本
print(torchvision.__version__)  #注意是双下划线

print(torch.version.cuda)  #注意是双下划线

print(torch.cuda.is_available())#如果cuda可用，则返回True

In [76]:
import numpy as np

batch_size=32
neighbor_size=40
num=20
neighbor=torch.randint(neighbor_size,[1,1])
for i in range(batch_size):
    item_select=torch.randint(neighbor_size,[1,1])
    while item_select.shape[1]<20:
        item=torch.randint(neighbor_size,[1,1])
        while item in item_select:
            item=torch.randint(neighbor_size,[1,1])
        item_select=torch.cat([item_select,item],dim=1)
    neighbor=torch.cat([neighbor,item_select],dim=0)

    

TypeError: randint() received an invalid combination of arguments - got (int, list, type=type), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool requires_grad)


In [77]:
int(torch.randint(neighbor_size,[1,1]))

9

In [69]:
item

tensor([[27]])

In [63]:
import numpy as np
np.random.randint(30,[1])

ValueError: low >= high

In [57]:
item

tensor([[2]])

In [34]:
a=torch.tensor(31)
a

tensor(31)

In [35]:
a in item_select

True